In [1]:
from py5paisa import FivePaisaClient
from py5paisa.order import Order, OrderType, Exchange, Bo_co_order
import time
import math
import pandas as pd
import warnings
from datetime import datetime

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
cred = {
    "APP_NAME": "5P56409084",
    "APP_SOURCE": "9217",
    "USER_ID": "pybeQPEbjju",
    "PASSWORD": "AESDZQGqyAa",
    "USER_KEY": "542GTEZguBbLERUXRZe26rLvfWU2X2KD",
    "ENCRYPTION_KEY": "l8sRTIsjl1JhUrz8X48XQJzKGI4k0gft",
}
client = FivePaisaClient(
    email="vasuappliedai@gmail.com", passwd="Vasudeva@1", dob="19981218", cred=cred
)
client.login()

 12:59:40 | Logged in!!


In [3]:
cred = {
    "APP_NAME": "5P57752064",
    "APP_SOURCE": "9322",
    "USER_ID": "5eRLuL81EDc",
    "PASSWORD": "mUG0xZduuO5",
    "USER_KEY": "BkVgrzXhLIUJbSYqqGIxNpyXcTtpSh0s",
    "ENCRYPTION_KEY": "pecRHVbBoroAavcjObdA6ITkK0ZmhSjq",
}
client = FivePaisaClient(
    email="chanderg16@gmail.com", passwd="5321@rvsA", dob="19720316", cred=cred
)
client.login()

 09:25:34 | Logged in!!


In [4]:
script_df = pd.read_csv("scripmaster-csv-format.csv")

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5352\3110652694.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  script_df = pd.read_csv("scripmaster-csv-format.csv")


In [5]:
current_hour = datetime.today().hour
current_minute = datetime.today().minute
current_hour,current_minute

(9, 27)

In [6]:
nifty_script_code = 999920000
banknifty_script_code = 999920005
given_date = datetime.today().strftime("%Y-%m-%d")
strikePrice = None
if(current_hour == 9 and current_minute <=30):
    strikePrice = client.historical_data("N", "C", banknifty_script_code, "1m", given_date, given_date).tail(1)["Open"].iloc[0]
else:
    strikePrice = client.historical_data("N", "C", banknifty_script_code, "1m", given_date, given_date)[
        "Open"
    ].loc[15]
strikePrice = round(strikePrice / 1000, 1) * 1000
print("Banknifty at 9:30am ", strikePrice)
expiry = "20220407"
symbol = "BANKNIFTY 07 Apr 2022"
call_symbol = symbol + " CE " + strikePrice.__str__() + "0"
put_symbol = symbol + " PE " + strikePrice.__str__() + "0"
call_script_code = script_df[script_df["FullName"]
                             == call_symbol].iloc[0]["Scripcode"].__int__()
put_script_code = script_df[script_df["FullName"]
                            == put_symbol].iloc[0]["Scripcode"].__int__()
call_script_code,put_script_code

Banknifty at 9:30am  38000.0


(42185, 42186)

In [8]:
req_list = [
    {
        "Exch": "N",
        "ExchType": "D",
        "Symbol": call_symbol,
        "Expiry": expiry,
        "StrikePrice": strikePrice,
        "OptionType": "CE",
    },
    {
        "Exch": "N",
        "ExchType": "D",
        "Symbol": put_symbol,
        "Expiry": expiry,
        "StrikePrice": strikePrice,
        "OptionType": "PE",
    },
]


In [9]:
def get_order_book():
    time.sleep(3)
    order_book =client.order_book()
    if order_book is None:
        for i in range(3):
            time.sleep(5)
            order_book =client.order_book()
            if order_book is not None:
                return order_book
    return order_book

def get_order_id(broker_order_id):
    order_book = get_order_book()
    for order in order_book:
        if(order["BrokerOrderId"] == broker_order_id):
            return order
    return None


def get_pending_order_by_script(script_code):
    order_book = get_order_book()
    for order in order_book:
        if(order["PendingQty"] > 0 and order["ScripCode"] == script_code and order["OrderStatus"] != "Cancelled"):
            return order
    return None


def place_co_bo(order_type, script_code, quantity, price, stop_loss):
    test_order = Bo_co_order(
        script_code,
        quantity,
        price,
        0,
        0,
        order_type,
        "N",
        "D",
        "p",
        stop_loss + 0.5,
        stop_loss,
    )
    order_broker_id = client.bo_order(test_order)["BrokerOrderIDNormal"]
    time.sleep(5)
    return get_order_id(order_broker_id)


def modify_sl_co_bo(script_code, quantity, stop_loss, exchange_id):
    test_order = test_order = Order(
        order_type="B",
        scrip_code=script_code,
        quantity=quantity,
        price=0,
        is_intraday=True,
        exchange="N",
        exchange_segment="D",
        exch_order_id=exchange_id,
        stoploss_price=stop_loss,
        is_stoploss_order=True,
    )
    return client.modify_order(test_order)


def get_last_traded_prices():
    market_data = client.fetch_market_feed([req_list[0]])["Data"]
    call_ltp = market_data[0]["LastRate"]
    market_data = client.fetch_market_feed([req_list[1]])["Data"]
    put_ltp = market_data[0]["LastRate"]
    return math.ceil(call_ltp), math.ceil(put_ltp)


def get_requested_ltp():
    market_data = client.fetch_market_feed(req_list)["Data"]
    return math.floor(market_data[0]["LastRate"])


def if_order_executed(exchange_id):
    order_book = get_order_book()
    for order in order_book:
        if(order["ExchOrderID"] == exchange_id and order["OrderStatus"] == "Fully Executed"):
            return True
    return False


def wait_for_order_execution():
    while_flag = True
    while while_flag:
        if if_order_executed(call_order_id):
            return call_script_code
        elif if_order_executed(put_order_id):
            return put_script_code
        else:
            time.sleep(60)



In [10]:
call_entry, put_entry = get_last_traded_prices()
call_stoploss, put_stoploss = math.ceil(call_entry*1.2), math.ceil(put_entry*1.2)

print("Put Entry and Stoploss ", put_entry, put_stoploss)
print("Call Entry and Stoploss ", call_entry, call_stoploss)

 09:27:25 | Success
 09:27:25 | Success
Put Entry and Stoploss  412 495
Call Entry and Stoploss  394 473


In [11]:
call_order_id = place_co_bo("S", call_script_code,
                            25, call_entry, call_stoploss)["ExchOrderID"]
put_order_id = place_co_bo("S", put_script_code, 25,
                           put_entry, put_stoploss)["ExchOrderID"]
call_order_id,put_order_id

 09:27:29 | 
 09:27:38 | 


('1400000005110192', '1600000004485799')

In [12]:
while(if_order_executed(call_order_id) == False):
    time.sleep(5)
while(if_order_executed(put_order_id) == False):
    time.sleep(5)

In [13]:
#uncomment this order is successful but it is unable to fetch the pending order
call_order_id="1500000006594652"
put_order_id="1700000006839716"

In [14]:
call_order_id = get_pending_order_by_script(call_script_code)["ExchOrderID"]
put_order_id = get_pending_order_by_script(put_script_code)["ExchOrderID"]


print("Call Order ID and PUT Order ID ", call_order_id, put_order_id)

TypeError: 'NoneType' object is not subscriptable

In [13]:
sl_hit_script = wait_for_order_execution()

order_id = None
script_code = None
req_list = None
if sl_hit_script == call_script_code:
    script_code = put_script_code
    order_id = put_order_id
    req_list = [{
        "Exch": "N",
        "ExchType": "D",
        "Symbol": put_symbol,
        "Expiry": expiry,
        "StrikePrice": strikePrice,
        "OptionType": "PE",
    }]
if sl_hit_script == put_script_code:
    script_code = call_script_code
    order_id = call_order_id
    req_list = [{
        "Exch": "N",
        "ExchType": "D",
        "Symbol": call_symbol,
        "Expiry": expiry,
        "StrikePrice": strikePrice,
        "OptionType": "CE",
    }]

In [15]:
# order_id = "1600000005411725"
# script_code = 42186
# req_list = [{
#         "Exch": "N",
#         "ExchType": "D",
#         "Symbol": call_symbol,
#         "Expiry": expiry,
#         "StrikePrice": strikePrice,
#         "OptionType": "CE",
#     }]


In [16]:
new_stop_loss = math.ceil(get_requested_ltp()*1.2)
modify_sl_co_bo(script_code, 25, new_stop_loss, order_id)
print("LTP & New Stoploss ", math.ceil(new_stop_loss*0.83),new_stop_loss)    

 09:44:22 | Success
LTP & New Stoploss  535 644


In [19]:
new_stop_loss

363

In [20]:
def wait_for_exit(script_code, order_id,new_stop_loss):
    while(if_order_executed(order_id) == False):
        df = client.historical_data(
            'N', 'D', script_code, '1m', given_date, given_date).tail(1)
        low_price = df["Low"].iloc[0]
        if low_price < new_stop_loss*0.75:
            new_stop_loss = math.ceil(low_price*1.2)
            print("Modified Stop Loss price, Low Stop loss price, Current Time ",new_stop_loss, low_price, datetime.now())
            modify_sl_co_bo(script_code, 25, new_stop_loss, order_id)
        time.sleep(60)

wait_for_exit(script_code, order_id,new_stop_loss)

 14:58:30 | Success


{'BrokerOrderID': 678115408,
 'ClientCode': '57752064',
 'Exch': 'N',
 'ExchOrderID': '1700000004077919',
 'ExchType': 'D',
 'LocalOrderID': 10,
 'Message': 'Success',
 'RMSResponseCode': 0,
 'ScripCode': 42095,
 'Status': 0,
 'Time': '/Date(1648751400000+0530)/'}